### Importing the libraries

In [30]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf

In [31]:
tf.__version__

'2.15.0'

## Part 1 - Data Preprocessing

### Importing the dataset

In [32]:
dataset = pd.read_csv('Churn_Modelling.csv')
x = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:,-1].values

In [33]:
# dataset.shape
print(x)
# print(y)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


### Encoding categorical data

Label Encoding the "Gender" column

In [34]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
x[:,2] = le.fit_transform(x[:,2])

In [35]:
print(x)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


One Hot Encoding the "Geography" column

In [36]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder',OneHotEncoder(),[1])],remainder='passthrough' )
x = np.array(ct.fit_transform(x))

In [37]:
print(x)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


### Splitting the dataset into the Training set and Test set

In [38]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)

### Feature Scaling

In [39]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Part 2 - Building the ANN

### Initializing the ANN

In [40]:
ann = tf.keras.models.Sequential()

### Adding the input layer and the first hidden layer

In [41]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

### Adding the second hidden layer

In [42]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

### Adding the output layer

In [43]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the ANN

### Compiling the ANN

In [44]:
ann.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

### Training the ANN on the Training set

In [45]:
ann.fit(X_train, Y_train, batch_size=32,epochs=150)

Epoch 1/150
250/250 [==============================] - 2s 3ms/step - loss: 0.5447 - accuracy: 0.7386
Epoch 2/150
250/250 [==============================] - 0s 2ms/step - loss: 0.4603 - accuracy: 0.7960
Epoch 3/150
250/250 [==============================] - 0s 2ms/step - loss: 0.4445 - accuracy: 0.7960
Epoch 4/150
250/250 [==============================] - 0s 2ms/step - loss: 0.4367 - accuracy: 0.7960
Epoch 5/150
250/250 [==============================] - 0s 2ms/step - loss: 0.4322 - accuracy: 0.7960
Epoch 6/150
250/250 [==============================] - 0s 2ms/step - loss: 0.4290 - accuracy: 0.7960
Epoch 7/150
250/250 [==============================] - 0s 2ms/step - loss: 0.4266 - accuracy: 0.7960
Epoch 8/150
250/250 [==============================] - 0s 2ms/step - loss: 0.4240 - accuracy: 0.8031
Epoch 9/150
250/250 [==============================] - 0s 2ms/step - loss: 0.4217 - accuracy: 0.8158
Epoch 10/150
250/250 [==============================] - 0s 2ms/step - loss: 0.4192 - accura

## Part 4 - Making the predictions and evaluating the model

### Predicting the result of a single observation

Use the ANN model to predict if the customer with the following informations will leave the bank:

Geography: France

Credit Score: 600

Gender: Male

Age: 40 years old

Tenure: 3 years

Balance: \$ 60000

Number of Products: 2

Does this customer have a credit card ? Yes

Is this customer an Active Member: Yes

Estimated Salary: \$ 50000

**Solution**

In [49]:
print(ann.predict(sc.transform([[1.0, 0.0, 0.0,600,1,40,3,60000,2,1,1,50000]])) > 0.5)

1/1 [==============================] - 0s 148ms/step
[[False]]


### Predicting the Test set results

In [50]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), Y_test.reshape(len(Y_test),1)),1))

63/63 [==============================] - 0s 2ms/step
[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


### Making the Confusion Matrix

In [52]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(Y_test, y_pred)
print(cm)
accuracy_score(Y_test,y_pred)

[[1499   96]
 [ 186  219]]


0.859